## Day 19 Aplenty

### Part 1

- We use a simple DFS.

In [39]:
# Part 1

from functools import lru_cache

file = open('day_19_input.txt').read().splitlines()

workflows = {}
break_point = file.index('')
ans = 0

@lru_cache
def execute_workflow(workflow: str, x: int, m: int, a:int, s:int):

    instructions = workflow.split(',')

    for instruction in instructions:
        condition, *next_workflow = instruction.split(':')

        if not next_workflow:
            return condition

        if eval(condition) and next_workflow:
            return next_workflow[0]

@lru_cache
def dfs(workflow: str, x: int, m: int, a: int, s: int):

    next_workflow = execute_workflow(workflow, x, m, a, s)
    if next_workflow == 'A':
        return x+m+a+s
    elif next_workflow == 'R':
        return 0

    return dfs(workflows[next_workflow], x, m, a, s)

for i, line in enumerate(file):
    if i < break_point:
        wf, instructions = line.strip('}').split('{')
        workflows[wf] = instructions

    elif i > break_point:
        x,m,a,s = line.strip('{').rstrip('}').split(',')

        x = int(x.split('=')[1])
        m = int(m.split('=')[1])
        a = int(a.split('=')[1])
        s = int(s.split('=')[1])

        ans += dfs(workflows['in'], x, m, a, s)

ans
# 263678

263678

### Part 2

- We use intervals for `x`, `m`, `a`, and `s` where the lower bound is inclusive and the upper bound is exclusive.
- For each condition in every workflow, for the general case, one interval `new_interval` will satisfy a condition, and there will be a `leftover` interval that doesn't satisfy the condition. We can then DFS using this `new_interval`, and pass on the `leftover` interval to the next condition(s).

In [32]:
# Part 2

from functools import lru_cache

file = open('day_19_input.txt').read().splitlines()

workflows = {}
break_point = file.index('')
ans = 0

def is_valid_range(lb: int, ub: int):

    if lb >= ub:
        return False
    return True

def generate_new_range(interval: tuple[int, int], sign: str, bound: int):

    new_interval = [interval[0], interval[1]]
    leftover = [interval[0], interval[1]]

    match sign:
        case '<':
            new_interval[1] = bound
            leftover[0] = bound
        case '>':
            new_interval[0] = bound + 1
            leftover[1] = bound + 1

    return tuple(new_interval), tuple(leftover)

@lru_cache
def dfs(workflow: str, x: tuple[int, int], m: tuple[int, int], a: tuple[int, int], s: tuple[int, int]):

    if workflow == 'A':
        return (x[1]-x[0])*(m[1]-m[0])*(a[1]-a[0])*(s[1]-s[0])
    elif workflow == 'R':
        return 0

    workflow = workflows[workflow]
    instructions = workflow.split(',')
    _sum = 0

    for instruction in instructions:
        condition, *next_workflow = instruction.split(':')

        if not next_workflow:
            _sum += dfs(condition, x, m, a, s)

        else:
            condvar, condsign, condbound = condition[0], condition[1], condition[2:]
            if condvar == 'x':
                new_interval, leftover = generate_new_range(x, condsign, int(condbound))
            elif condvar == 'm':
                new_interval, leftover = generate_new_range(m, condsign, int(condbound))
            elif condvar == 'a':
                new_interval, leftover = generate_new_range(a, condsign, int(condbound))
            elif condvar == 's':
                new_interval, leftover = generate_new_range(s, condsign, int(condbound))
            if is_valid_range(new_interval[0], new_interval[1]):
                _sum += dfs(next_workflow[0], new_interval if condvar == 'x' else x, new_interval if condvar == 'm' else m, new_interval if condvar == 'a' else a, new_interval if condvar == 's' else s)
            if is_valid_range(leftover[0], leftover[1]):
                match condvar:
                    case 'x':
                        x = leftover
                    case 'm':
                        m = leftover
                    case 'a':
                        a = leftover
                    case 's':
                        s = leftover

    return _sum 


for i, line in enumerate(file):
    if i < break_point:
        wf, instructions = line.strip('}').split('{')
        workflows[wf] = instructions
    else:
        break

ans = dfs('in', (1, 4001), (1, 4001), (1, 4001), (1, 4001))
ans
#125455345557345

125455345557345